# 整体架构

```mermaid
graph TB
    subgraph "基础库层 (BaseLibs)"
        A1[Share<br/>基础数据结构与接口]
        A2[WTSUtils<br/>第三方组件封装]
        A3[WTSTools<br/>通用工具库]
        A4[Includes<br/>头文件定义]
    end
    
    subgraph "数据层 (DataKit)"
        B1[WtDtCore<br/>数据组件核心]
        B2[WtDtPorter<br/>数据C接口导出]
        B3[QuoteFactory<br/>数据落地程序]
        B4[WtDataStorage<br/>数据读取组件]
        B5[WtDtServo<br/>数据伺服器]
    end
    
    subgraph "行情解析层 (Parsers)"
        C1[ParserCTP<br/>CTP行情解析]
        C2[ParserXTP<br/>XTP行情解析]
        C3[ParserUDP<br/>UDP广播解析]
        C4[ParserFemas<br/>飞马行情解析]
        C5[ParserOther<br/>其他行情解析器...]
    end
    
    subgraph "交易引擎层 (Engines)"
        D1[WtCore<br/>实盘交易核心]
        D2[WtBtCore<br/>回测引擎核心]
        D3[WtUftCore<br/>超高频引擎核心]
    end
    
    subgraph "策略管理层"
        E1[CtaStrategyMgr<br/>CTA策略管理器]
        E2[HftStrategyMgr<br/>HFT策略管理器]
        E3[SelStrategyMgr<br/>SEL策略管理器]
        E4[UftStrategyMgr<br/>UFT策略管理器]
    end
    
    subgraph "交易通道层 (Traders)"
        F1[TraderCTP<br/>CTP交易通道]
        F2[TraderXTP<br/>XTP交易通道]
        F3[TraderMocker<br/>本地仿真撮合]
        F4[TraderOther<br/>其他交易通道...]
    end
    
    subgraph "执行层"
        G1[WtExeFact<br/>执行器工厂]
        G2[WtRiskMonFact<br/>风控单元工厂]
        G3[执行单元<br/>算法交易执行]
    end
    
    subgraph "接口层 (Porters)"
        H1[WtPorter<br/>实盘C接口]
        H2[WtBtPorter<br/>回测C接口]
        H3[WtDtPorter<br/>数据C接口]
    end
    
    subgraph "运行程序层 (Runners)"
        I1[WtRunner<br/>实盘运行程序]
        I2[WtBtRunner<br/>回测运行程序]
        I3[WtUftRunner<br/>超高频运行程序]
        I4[QuoteFactory<br/>数据落地程序]
    end
    
    subgraph "辅助工具层 (Tools)"
        J1[CTPLoader<br/>合约加载器]
        J2[TraderDumper<br/>交易数据落地]
        J3[LoaderRunner<br/>加载器运行程序]
    end
    
    %% 依赖关系
    A1 --> B1
    A1 --> C1
    A1 --> D1
    A1 --> F1
    A2 --> B1
    A3 --> B1
    A4 --> B1
    
    B1 --> D1
    B1 --> D2
    B2 --> H3
    B3 --> I4
    
    C1 --> B1
    C2 --> B1
    C3 --> B1
    
    D1 --> E1
    D1 --> E2
    D1 --> E3
    D2 --> E1
    D2 --> E2
    D3 --> E4
    
    E1 --> G1
    E2 --> G1
    E3 --> G1
    E4 --> G1
    
    F1 --> D1
    F2 --> D1
    F3 --> D1
    
    G1 --> H1
    G2 --> H1
    
    H1 --> I1
    H2 --> I2
    
    J1 --> I3
    J2 --> I1
```

## 基础库层 (BaseLibs)
- **Share/**: 核心基础库，定义了整个框架的基础数据结构、基础对象和所有接口
  - `WTSObject.hpp`: 基础对象类
  - `WTSDataDef.hpp`: 数据结构定义
  - `WTSTradeDef.hpp`: 交易相关定义
  - `SpinMutex.hpp`: 高性能自旋锁
  - `threadpool/`: 线程池实现

- **WTSUtils/**: 第三方组件封装库
  - `yamlcpp/`: YAML 配置文件解析
  - `zstdlib/`: 数据压缩库
  - `lmdb/`: 高性能键值数据库
  - `WTSCfgLoader`: 配置加载器

- **WTSTools/**: 通用工具库
  - `WTSLogger`: 日志系统
  - `WTSBaseDataMgr`: 基础数据管理
  - `WTSDataFactory`: 数据工厂
  - `WTSHotMgr`: 主力合约管理

## 数据层 (DataKit)
- **WtDtCore/**: 数据组件核心，包含数据接入落地的核心逻辑
- **WtDataStorage/**: 数据读取组件，用于读取 WT 自有格式的行情数据
- **WtDataStorageAD/**: 数据落地组件，将行情数据落地到文件
- **WtDtServo/**: 数据伺服器，提供实时数据随机访问接口
- **QuoteFactory/**: 数据落地程序入口

## 行情解析层 (Parsers)
支持多种行情接口：
- **期货类**: ParserCTP、ParserCTPMini、ParserFemas、ParserYD、ParserXeleSkt
- **期权类**: ParserCTPOpt、ParserMA、ParserAres  
- **股票类**: ParserHuaX、ParserXTP、ParserOES
- **通用类**: ParserUDP（UDP广播）、ParserShm（共享内存）

## 交易引擎层 (Engines)

**WtCore (实盘交易核心)**
- `WtEngine`: 交易引擎基类
- `WtCtaEngine`: CTA 同步策略引擎
- `WtHftEngine`: HFT 高频策略引擎  
- `WtSelEngine`: SEL 异步策略引擎
- `WtExecMgr`: 执行管理器
- `WtDtMgr`: 数据管理器

**WtBtCore (回测引擎核心)**
- `HisDataReplayer`: 历史数据回放器
- `MatchEngine`: 撮合引擎
- `CtaMocker`: CTA 策略回测模拟器
- `HftMocker`: HFT 策略回测模拟器
- `SelMocker`: SEL 策略回测模拟器

**WtUftCore (超高频引擎核心)**
- `WtUftEngine`: UFT 超高频引擎
- `UftStrategyMgr`: UFT 策略管理器
- `ShareManager`: 共享数据管理器

## 交易通道层 (Traders)
支持多种交易接口：
- **期货类**: TraderCTP、TraderCTPMini、TraderFemas、TraderYD
- **期权类**: TraderCTPOpt、TraderMAOpt、TraderAresClt
- **股票类**: TraderXTP、TraderXTPXAlgo、TraderATP、TraderOES、TraderHuaX
- **仿真类**: TraderMocker（本地仿真撮合）

## 执行与风控层
- **WtExeFact/**: 执行器工厂
  - 内置多种算法交易执行单元（TWAP、VWAP、MinImpact等）
- **WtRiskMonFact/**: 风控单元工厂
  - 提供组合盘资金风控功能

## 接口层 (Porters)
- **WtPorter/**: 实盘交易框架 C 接口导出库
- **WtBtPorter/**: 回测框架 C 接口导出库  
- **WtDtPorter/**: 数据组件 C 接口导出库
- **WtExecMon/**: 独立执行器 C 接口导出库